## Tune Regression 

In [1]:
# DAL ToolBox
# version 1.0.767

source("https://raw.githubusercontent.com/cefet-rj-dal/daltoolbox/main/jupyter.R")

#loading DAL
load_library("daltoolbox") 

Loading required package: daltoolbox

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


Attaching package: ‘daltoolbox’


The following object is masked from ‘package:base’:

    transform




### Dataset for classification

In [2]:
iris <- datasets::iris
head(iris)

#extracting the levels for the dataset
slevels <- levels(iris$Species)
slevels

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


[1] "setosa"     "versicolor" "virginica"

## Building samples (training and testing)

In [3]:
# preparing dataset for random sampling
set.seed(1)
sr <- sample_random()
sr <- train_test(sr, iris)
iris_train <- sr$train
iris_test <- sr$test

tbl <- rbind(table(iris[,"Species"]),
             table(iris_train[,"Species"]),
             table(iris_test[,"Species"]))
rownames(tbl) <- c("dataset", "training", "test")
head(tbl)

,setosa,versicolor,virginica
dataset,50,50,50
training,39,38,43
test,11,12,7


### Training

In [4]:
tune <- cla_tune(cla_svm("Species", slevels))
ranges <- list(epsilon=seq(0,1,0.2), cost=seq(20,100,20), kernel = c("linear", "radial", "polynomial", "sigmoid"))

model <- fit(tune, iris_train, ranges)

### Model adjustment

In [5]:
train_prediction <- predict(model, iris_train)

iris_train_predictand <- adjust_class_label(iris_train[,"Species"])
train_eval <- evaluate(model, iris_train_predictand, train_prediction)
print(train_eval$metrics)

   accuracy        f1 sensitivity specificity precision    recall
1 0.9888889 0.9833333   0.9833333   0.9916667 0.9833333 0.9833333


### Test

In [6]:
test_prediction <- predict(model, iris_test)

iris_test_predictand <- adjust_class_label(iris_test[,"Species"])
test_eval <- evaluate(model, iris_test_predictand, test_prediction)
print(test_eval$metrics)

   accuracy        f1 sensitivity specificity precision    recall
1 0.9555556 0.9333333   0.9333333   0.9666667 0.9333333 0.9333333


### Options for other models

In [7]:
#knn
ranges <- list(k=1:20)

#mlp
ranges <- list(size=1:10, decay=seq(0, 1, 0.1))

#rf
ranges <- list(mtry=1:3, ntree=1:10)